In [ ]:
# also explore: https://scikit-learn.org/stable/modules/feature_selection.html

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn import metrics

%matplotlib inline

In [42]:
# TRAIN_FILE = './data/tony-finau-round-course-train.csv'
# TEST_FILE = './data/tony-finau-round-course-test.csv'

TRAIN_FILE = './data/2015-2017-round-course-train.csv'
TEST_FILE = './data/2015-2017-round-course-test.csv'

In [43]:
def parse_teetime_hour(val):
    return val[0:val.find(':')]

def load_and_process_data(file):
    df = pd.read_csv(file, index_col=None)
    
    # Convert string fields (course data) to numeric
    label_encoder = LabelEncoder()
    df['CourseFairwayFirmness'] = label_encoder.fit_transform(df['CourseFairwayFirmness'])
    df['CourseGreenFirmness'] = label_encoder.fit_transform(df['CourseGreenFirmness'])
    df['CourseGreenGrass'] = label_encoder.fit_transform(df['CourseGreenGrass'])
    df['CourseFairwayGrass'] = label_encoder.fit_transform(df['CourseFairwayGrass'])
    df['CourseTeeGrass'] = label_encoder.fit_transform(df['CourseTeeGrass'])
    df['CourseRoughGrass'] = label_encoder.fit_transform(df['CourseRoughGrass'])

    # Convert TeeTime into just the hour
    df['TeeTime'] = df['TeeTime'].apply(parse_teetime_hour)
    
    # Convert -1 for no attempts to mean value??
    # imputer = preprocessing.Imputer(missing_values='-1', strategy='mean', axis=0)
    # imputer1 = imputer.fit_transform(df['Approach50-75'].values.reshape(-1,1))
    # df['Approach50-75'] = imputer1
    
    # Drop score related columns
    df.drop(
        columns=['ScoreEagles', 'ScoreBirdies', 'ScorePars', 'ScoreBogeys', 'ScoreDoubles', 'ScoreOthers', 'ScoreOverPar'],
        axis=1,
        inplace=True
    )
    
    # Drop strokes-gained related columns (calculated after the round)
    df.drop(
        columns=['SGPutting', 'SGTeeToGreen', 'SGTotal', 'SGOffTheTee', 'SGApproach', 'SGAroundTheGreen'],
        axis=1,
        inplace=True
    )
    
    # Drop end of round/event position
    df.drop(
        columns=['EndOfRoundFinishPosition', 'EndOfEventFinishPosition'],
        axis=1,
        inplace=True
    )
    
    return df

def split_data(df):
    # return X, y
    return df[df.columns[1:]], df[df.columns[0]]

In [44]:
df_train = load_and_process_data(TRAIN_FILE)
df_test = load_and_process_data(TEST_FILE)
X_train, y_train = split_data(df_train)
X_test, y_test = split_data(df_test)

In [45]:
X = X_train.values
y = y_train.values

X = StandardScaler().fit_transform(X)
X

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-1.21557683, -1.14019838,  1.02578388, ...,  0.3153713 ,
        -0.37037008,  0.32336365],
       [-1.21557683, -1.14019838,  1.02578388, ...,  0.3153713 ,
        -0.37037008,  0.32336365],
       [-1.21557683, -1.14019838,  1.02578388, ...,  0.3153713 ,
        -0.37037008,  0.32336365],
       ...,
       [ 1.21838416,  1.61240602, -1.33148482, ...,  0.7887544 ,
         0.36885292,  0.43184605],
       [ 1.21838416,  1.61240602, -1.33148482, ...,  0.7887544 ,
         0.36885292,  0.43184605],
       [ 1.21838416,  1.61240602, -1.33148482, ...,  0.7887544 ,
         0.36885292,  0.43184605]])

In [46]:
n_components = 10
cols = ['pc' + str(i+1) for i in range(n_components)]

pca = PCA(n_components=n_components)
principalComponents = pca.fit_transform(X)
df_principal = pd.DataFrame(data=principalComponents, columns=cols)
df_principal.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,-1.178702,1.981185,3.410324,-0.232543,1.526125,-2.945778,-0.528402,-0.970089,0.568414,-0.278267
1,-1.471551,2.556305,-1.381781,-2.049728,0.388242,-1.909400,-3.497386,0.743764,2.937712,0.073706
2,-0.013205,1.799104,3.678729,-1.031677,1.764268,-1.621545,-0.649825,-0.967563,-1.098659,0.074878
3,-2.831524,3.072544,-1.263909,-2.862413,0.149422,0.098846,-0.347760,-0.844354,0.973776,-0.246929
4,-2.747055,3.138805,-2.239306,-3.087656,0.390938,-0.259356,-0.903145,-0.536206,0.622093,0.368298


In [47]:
df_final = pd.concat([df_principal, y_train], axis = 1)
df_final.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,Score
0,-1.178702,1.981185,3.410324,-0.232543,1.526125,-2.945778,-0.528402,-0.970089,0.568414,-0.278267,77
1,-1.471551,2.556305,-1.381781,-2.049728,0.388242,-1.909400,-3.497386,0.743764,2.937712,0.073706,73
2,-0.013205,1.799104,3.678729,-1.031677,1.764268,-1.621545,-0.649825,-0.967563,-1.098659,0.074878,72
3,-2.831524,3.072544,-1.263909,-2.862413,0.149422,0.098846,-0.347760,-0.844354,0.973776,-0.246929,69
4,-2.747055,3.138805,-2.239306,-3.087656,0.390938,-0.259356,-0.903145,-0.536206,0.622093,0.368298,70


In [48]:
pca.explained_variance_ratio_

array([0.2664863 , 0.05918143, 0.04342693, 0.03727715, 0.03006946,
       0.02661499, 0.02077027, 0.02000563, 0.01747479, 0.01717817])

In [49]:
#plt.matshow(df_train.corr())
corr = df_train.corr()
corr.style.background_gradient()

In [50]:
df_corr = pd.DataFrame(corr['Score'], index=None, columns=['Score'])
df_corr.reset_index(level=0, inplace=True)
df_corr.columns = ['Name', 'Score']
df_corr.sort_values(by='Score', ascending=False, inplace=True)
df_corr.head(11)

,Name,Score
0,Score,1.000000
38,PuttingTotal,0.407784
41,Putting3,0.290498
72,CoursePar5TotalScorecardYardage,0.237212
69,CoursePar5Holes,0.226110
94,CourseTotalScorecardYardage,0.224211
3,TotalPar,0.216548
24,AvgProximityToHole,0.207023
25,AvgProximityFairway,0.206537
37,ScramblingSandProximity,0.157350
